In [46]:
import os
import glob
import re 
from xml.dom import minidom
import pandas as pd
import tqdm

# read the data

In [2]:
os.system('tar -xvf  ./data/NLMCXR_reports.tgz -C ./data/')

0

In [3]:
allFiles = glob.glob(f'./data/**/*.xml', recursive=True)

In [4]:
len(allFiles)

3955

# Convert XML to csv

In [53]:
def getfilename(xml_file):
    x = re.search("[ \w-]+?(?=\.)", xml_file)
    return x.group()

def get_report(xml_file):
    #get radiology report text
    data = {}
    mydoc = minidom.parse(xml_file)
    elements = mydoc.getElementsByTagName('AbstractText')
    for element in elements:
        txt = ""
        if element.firstChild != None:
            txt = element.firstChild.data
        data[element.attributes['Label'].value] = txt
        
    return data

def get_labels(xml_file):
    data = {}
    mydoc = minidom.parse(xml_file)
    elements = mydoc.getElementsByTagName('major')
    txt = []
    for element in elements:
        if element.firstChild != None:
            txt.append(element.firstChild.data)
    data['expert_labels'] = txt
    
    elements = mydoc.getElementsByTagName('automatic')
    txt = []
    for element in elements:
        if element.firstChild != None:
            txt.append(element.firstChild.data)
    data['manual_labels'] = txt
    
    return data

def convert_xml2csv(allFiles):
    columns = ["fileNo", "COMPARISON", "INDICATION", "FINDINGS", "IMPRESSION", "expert_labels", "manual_labels"]
    xml_df = pd.DataFrame(columns = columns)
    for xml_file in allFiles:
#         print(xml_file)
        row = {}
        row['fileNo'] = getfilename(xml_file)

        reportDic = get_report(xml_file)
        labelsDic = get_labels(xml_file)

        row = {**row ,**reportDic, **labelsDic}
        xml_df = xml_df.append(row,ignore_index=True)
    return xml_df

xml_df = convert_xml2csv(allFiles)
xml_df.head()

,fileNo,COMPARISON,INDICATION,FINDINGS,IMPRESSION,expert_labels,manual_labels
0,881,None available.,XXXX-year-old XXXX with dyspnea.,The lungs are without focal air space opacity....,No acute cardiopulmonary abnormality.,[normal],[]
1,1734,None.,Back pain,Heart size and mediastinal contour are normal....,No acute cardiopulmonary process.,[normal],[]
2,306,None.,,The lungs are clear. Heart size is normal. No ...,Clear lungs. No acute cardiopulmonary abnormal...,[normal],[]
3,3188,"Chest x-XXXX, XXXX, XXXX.",This is a XXXX-year-old male patient with vomi...,,The cardiac silhouette appears be at upper lim...,"[Cardiac Shadow/enlarged/borderline, Mediastin...",[]
4,1951,,,Cardiomediastinal silhouette is normal. Pulmon...,No acute cardiopulmonary disease.,[normal],[]
